---
# 라이브러리 불러오기
----

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from datetime import date, datetime, timedelta
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

- data_history = 액티브 휴폐업이력 Sheet + 휴폐업 휴폐업이력 Sheet
  
- data = 액티브 기업개요_외감 Sheet + 휴폐업 기업개요 Sheet 

- finance = 재무 데이터 수정 본

---
# 데이터셋 불러오기
----

In [2]:
data_history = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/휴폐업이력_합본.csv')
data = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/엑티브_휴폐업_합본.csv')
finance = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/재무데이터_수정.csv')
del data_history['Unnamed: 0']
del data['Unnamed: 0']
del finance['Unnamed: 0']

In [3]:
print('액티브 + 휴폐업 데이터셋 크기:',data.shape)
print('휴폐업 이력이 있는 데이터셋 크기:',data_history.shape)
print('재무 데이터셋 크기:',finance.shape)

액티브 + 휴폐업 데이터셋 크기: (35721, 31)
휴폐업 이력이 있는 데이터셋 크기: (11667, 9)
재무 데이터셋 크기: (109142, 53)


---
# 0. dataset 구성
---
- 예시는 아래와 같이 구성

|사업자번호||섹터||년도||기업매출||~~~||~~~||~~~||~~~|
|---||---||---||---||---||---||---||---|
|1234||A||20210||10000||10000||10000||10000||10000|
|1234||A||2021||20000||10000||10000||10000||10000|
|5678||B||2018||5000||10000||10000||10000||10000|
|5678||B||2020||17000||10000||10000||10000||10000|

---
### 0-1. 결산년월의 월이 연말(=1231)로 설정되지 않은 값 제거
----

In [4]:
finance['결산년월'] = finance['결산년월'].astype(str)

In [5]:
print('데이터의 수 :',len(finance))
print('결산년월이 월말(=1231)인 값의 수 :',len(finance[finance['결산년월'].str.contains('1231')]))
print('결산년월에서 제거되는 데이터 수 :',len(finance) - len(finance[finance['결산년월'].str.contains('1231')]))

데이터의 수 : 109142
결산년월이 월말(=1231)인 값의 수 : 105141
결산년월에서 제거되는 데이터 수 : 4001


In [6]:
finance = finance[finance['결산년월'].str.contains('1231')]
finance = finance.reset_index(drop = True)
len(finance)


105141

---
### 표준산업분류(=섹터) 분류 표
----

GCIS 산업분류체계표 기준 11개를 기준으로 표준산업분류(총 21개)를 분류

1. 에너지 [Energy] = 전기,가스, 증기 및 공기 조절 공급업 (D)  

2. 소재 [Materials] = 수도,하수 및 폐기물 처리, 원료 재생업 (E)
3. 산업재 [Industrials] = 광업 (B) / 제조업 (C) / 건설업 (F) / 운수 및 창고업 (H)
4. 자유 소비재 [Discretionary] = 협회 및 단체, 수리 및 기타 개인 서비스업 (S) / 예술, 스포츠 및 여가관련 서비스업 (R) / 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동 (T) / 교육 서비스업 (P) / 도매 및 소매업 (G)
5. 필수 소비재 [Staples] = 숙박 및 음식점업 (I) / 농업,임업 및 어업 (A)
6. 건강관리 [Health] = 보건업 및 사회복지 서비스업 (Q)
7. 금융 [Financials] = 금융 및 보험업 (K)
8. 정보기술 [Technology] = 정보통신업 (J) / 전문, 과학 및 기술 서비스업 (M)
9. 유틸리티 [Utilities] = 공공 행정, 국방 및 사회보장 행정 (O), 국제 및 외국기관 (U)
10. 부동산 [Estate] = 사업시설 관리, 사업 지원 및 임대 서비스업 (N) / 부동산업 (L)
11. 커뮤니케이션 서비스 [Communication] = 
---

In [7]:
##### 약 11분 소요 #####

dataset = finance[['사업자번호','결산년월','휴폐업이력']]

dataset['파일분류'] = None
dataset['표준산업분류'] = None

for i in range(len(dataset)):
  if dataset['사업자번호'][i] in data['사업자등록번호'].tolist() :
    dataset['파일분류'][i] = data[(data['사업자등록번호'] == dataset['사업자번호'][i])]['파일분류'].item()
    dataset['표준산업분류'][i] = data[(data['사업자등록번호'] == dataset['사업자번호'][i])]['표준산업분류'].item()

In [8]:
finance['결산년월'].value_counts()

20181231    29700
20191231    26360
20211231    24583
20201231    24498
Name: 결산년월, dtype: int64

---
### 0-2. 결산년월의 표기를 년도로만 표시
----

In [9]:
for i in range(len(dataset)):
  if dataset['결산년월'][i] == '20181231':
    dataset['결산년월'][i] = '2018'
  elif dataset['결산년월'][i] == '20191231':
    dataset['결산년월'][i] = '2019'
  elif dataset['결산년월'][i] == '20201231':
    dataset['결산년월'][i] = '2020'
  else:
    dataset['결산년월'][i] = '2021'

---
# 1. 재무데이터를 활용한 DCF 기업가치
---

    세후영업이익(NOPLAT) -> EBIT(영업이익) - TAX(법인세 비용)
    - 운전자본비용(NWC) -> 매출채권 + 재고자산 - 매입채무
    + 비현금비용(감가상각비)
    - 자본적 지출(CAPEX) -> 비유동자산
    ------------------------------------------------------------
    FCFF

- finance 데이터를 기반으로 dataset을 형성 했기 때문에 "기업가치"에 관련된 값을 concat 해주겠다.

In [10]:
dataset = pd.concat([dataset,finance['기업가치']], axis = 1)
dataset.isnull().sum()

사업자번호         0
결산년월          0
휴폐업이력     98306
파일분류          0
표준산업분류      181
기업가치      57246
dtype: int64

----
# 2. 특허 출원 여부 및 출원 수
----

---
# 3. 섹터별 주가 현황 데이터
---

---
# 4. 기업시장지수
---

---
# 5. 기업할당지수
----

- 매출액과 자산총계 추가

In [11]:
# 기존 재무데이터의 매출액 합치기
dataset = pd.concat([dataset,finance[['매출액','자산총계','영업이익（손실）','당기순이익(손실)']]], axis = 1)

# 데이터 타입 변환
dataset[['매출액','기업가치','자산총계','영업이익（손실）','당기순이익(손실)']] = dataset[['매출액','기업가치','자산총계','영업이익（손실）','당기순이익(손실)']].fillna(0).astype(int).replace({0:None})

- 년도별 섹터분류 후 총 매출액의 합 계산

In [12]:
##### 약 47분 소요 #####

dataset['기업할당지수'] = None

sector = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()['2018'].to_dict().keys()

for i in range(len(dataset)):
  for j in sector:
    try:
      if (dataset['결산년월'][i] == '2018') & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()['2018'].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j]) * 100

      elif (dataset['결산년월'][i] == '2019') & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()['2019'].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j]) * 100

      elif (dataset['결산년월'][i] == '2020') & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()['2020'].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j]) * 100

      elif (dataset['결산년월'][i] == '2021') & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()['2021'].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j])* 100
    except:
      pass 

In [13]:
dataset[(dataset['표준산업분류'] == 'Estate') & (dataset['결산년월'] == '2019')]['기업할당지수'].sum()

100.00000000000003

---
# 6. 기업 성장률 지표
---

---
### 6-1 연평균 성장률 (CAGR)
---

- (최신연도 매출액/설립연도 매출액)^(1/(최신연도-설립연도))-1

In [14]:
# 기존 재무데이터에서 만들어 놓은 매출액 성장률 합치기
dataset = pd.concat([dataset,finance['매출액성장률']], axis = 1)
dataset.isnull().sum()

사업자번호         0
결산년월          0
휴폐업이력     98306
파일분류          0
표준산업분류      181
기업가치      57246
매출액        5222
자산총계        808
기업할당지수     5394
매출액성장률    38444
dtype: int64

---
### 6-2 기업 안정성 평가
---

- **매출 성장률** = (당해년도 매출액 / 직전년도 매출액) x 100 - 100
- **총자산 성장률** = (당해년도 총자산 / 직전년도 총자산) x 100 - 100

In [15]:
############ 사업자번호가 중복되는 값의 인덱스를 변환하기 위한 코드 ############

# 사업자번호의 중복값의 시작값은 True
a = dataset['사업자번호'].duplicated()

# 사업자번호 전체 인덱스 값 생성
b = list(range(len(a)))

for i in range(len(a)):
  if a[i] == True: # True 라는 것은 사업자번호가 중복되는 값의 시작점이며 다음 False가 나올기 전 까지 중복된 값임
    b.remove(i)

print('재무제표 전체 데이터 수:',len(dataset))
print('중복을 제외한 사업자번호 수:',len(b))

재무제표 전체 데이터 수: 105141
중복을 제외한 사업자번호 수: 34751


In [16]:
##### 약 1분 소요 #####

dataset['매출 성장률'] = None

for i in range(len(b)):
  num = b[i]
  if num == 105137 :
    num2 = 105141
  else: 
    num2 = b[i+1]
  count = num2 - num

  for j in range(count):
    try:
      if j+1 != count:
        dataset['매출 성장률'][num + j + 1] = round(((dataset['매출액'][num + j + 1] - dataset['매출액'][num + j]) / dataset['매출액'][num + j]) * 100 ,3)
      else :
        pass
    except:
      pass

In [17]:
##### 약 1분 소요 #####

dataset['총자산 성장률'] = None

for i in range(len(b)):
  num = b[i]
  if num == 105137 :
    num2 = 105141
  else: 
    num2 = b[i+1]
  count = num2 - num

  for j in range(count):
    try:
      if j+1 != count:
        dataset['총자산 성장률'][num + j + 1] = round(((dataset['자산총계'][num + j + 1] - dataset['자산총계'][num + j]) / dataset['자산총계'][num + j]) * 100 ,3)
      else :
        pass
    except:
      pass

---
# 7. 기업의 존속기간별 구간
---

- 9 = 1년 이하 
- 8 = 1년 ~ 2년 
- 7 = 2년 ~ 3년 
- 6 = 3년 ~ 4년 
- 5 = 4년 ~ 5년 
- 4 = 5년 ~ 7년 
- 3 = 7년 ~ 10년 
- 2 = 10년 ~ 15년 
- 1 = 15년 ~ 20년 
- 0 = 20년 이상 



In [18]:
##### 약 9분 소요 #####

dataset['존속기간'] = None

for i in range(len(dataset)):
  if dataset['사업자번호'][i] in data['사업자등록번호'].tolist() :
    dataset['존속기간'][i] = data[(data['사업자등록번호'] == dataset['사업자번호'][i])]['존속기간'].item()

# 데이터 타입 변환
dataset['존속기간'] = dataset['존속기간'].fillna(0).astype(int).replace({0:None})

In [26]:
##### 약 1분 소요 #####

for i in range(len(dataset)):
  try:
    if dataset['결산년월'][i] == "2020":
      dataset['존속기간'][i] = dataset['존속기간'][i] - 300
    elif dataset['결산년월'][i] == "2019":
      dataset['존속기간'][i] = dataset['존속기간'][i] - 665
    elif dataset['결산년월'][i] == "2018":
      dataset['존속기간'][i] = dataset['존속기간'][i] - 1030
  except:
    pass 

In [195]:
index = dataset[dataset['존속기간'] <= 365].index
dataset['존속기간'][index] = 9
    
index = dataset[(dataset['존속기간'] > 365) & (dataset['존속기간'] <= 730)].index
dataset['존속기간'][index] = 8

index = dataset[(dataset['존속기간'] > 730) & (dataset['존속기간'] <= 1095)].index
dataset['존속기간'][index] = 7

index = dataset[(dataset['존속기간'] > 1095) & (dataset['존속기간'] <= 1460)].index
dataset['존속기간'][index] = 6

index = dataset[(dataset['존속기간'] > 1460) & (dataset['존속기간'] <= 1825)].index
dataset['존속기간'][index] = 5

index = dataset[(dataset['존속기간'] > 1825) & (dataset['존속기간'] <= 2555)].index
dataset['존속기간'][index] = 4

index = dataset[(dataset['존속기간'] > 2555) & (dataset['존속기간'] <= 3650)].index
dataset['존속기간'][index] = 3

index = dataset[(dataset['존속기간'] > 3650) & (dataset['존속기간'] <= 5475)].index
dataset['존속기간'][index] = 2

index = dataset[(dataset['존속기간'] > 5475) & (dataset['존속기간'] <= 7300)].index
dataset['존속기간'][index] = 1

index = dataset[(dataset['존속기간'] > 7300)].index
dataset['존속기간'][index] = 0

In [162]:
dataset.isnull().sum()

사업자번호            0
결산년월             0
휴폐업이력            0
파일분류             0
표준산업분류         181
기업가치         57246
매출액           5222
자산총계           808
기업할당지수        5394
매출액성장률       38444
매출 성장률       38712
총자산 성장률      35317
존속기간          1855
영업이익（손실）       910
당기순이익(손실)      866
dtype: int64

---
# 8. Target값 생성
----

In [157]:
dataset['휴폐업이력'] = 0

check = data_history[(data_history['CLSBZ_GB'] == 2) | (data_history['CLSBZ_GB'] == 3)]
check = check.reset_index(drop = True)

for i in range(len(check)):
  if check['BIZ_NO'][i] in dataset['사업자번호'].tolist():
    if (type(check['STRT_DATE'][i]) == str) == False:
      if check['END_DATE'][i][:4] == '2019' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2018')].index
        dataset['휴폐업이력'][number] = 1
      
      elif check['END_DATE'][i][:4] == '2020' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2019')].index
        dataset['휴폐업이력'][number] = 1
      
      elif check['END_DATE'][i][:4] == '2021' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2020')].index
        dataset['휴폐업이력'][number] = 1
      elif check['END_DATE'][i][:4] == '2022' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2021')].index
        dataset['휴폐업이력'][number] = 1             
    
    else :
      if check['STRT_DATE'][i][:4] == '2019' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2018')].index
        dataset['휴폐업이력'][number] = 1
      
      elif check['STRT_DATE'][i][:4] == '2020' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2019')].index
        dataset['휴폐업이력'][number] = 1
      
      elif check['STRT_DATE'][i][:4] == '2021' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2020')].index
        dataset['휴폐업이력'][number] = 1
      elif check['STRT_DATE'][i][:4] == '2022' :
        number = dataset[(dataset['사업자번호'] == check['BIZ_NO'][i]) & (dataset['결산년월'] == '2021')].index
        dataset['휴폐업이력'][number] = 1           